<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/Simple_ABF_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Simple ABF tools

import numpy as np
import pyabf
import matplotlib
import matplotlib.pyplot as plt

def protocol_baseline_and_stim(abf):
    'Return two boolean arrays, distiguishing holding I/V and electrical stimuli'
    # use command signal variance to determine stimulus periods
    commands = []
    for s in abf.sweepList:
        abf.setSweep(sweepNumber=s)
        commands.append(abf.sweepC)
    commands = np.stack(commands)
    
    std = np.std(commands, axis=0)
    is_base = std==0
    is_stim = np.logical_not(is_base)
    return is_base, is_stim

def protocol_baseline_and_stim(abf):
    'Return two boolean arrays, distiguishing holding I/V and electrical stimuli'
    # use command signal variance to determine stimulus periods
    commands = []
    for s in abf.sweepList:
        abf.setSweep(sweepNumber=s)
        commands.append(abf.sweepC)
    commands = np.stack(commands)
    
    std = np.std(commands, axis=0)
    is_base = std==0
    is_stim = np.logical_not(is_base)
    return is_base, is_stim

def plot_sweeps_and_command(abf,figsize = [8,2],windows=[]):
    'Plot an abf file with sweeps and command'
    'also attempts to calibrate telegraph offset when a Ch1 is a secondary ouput of the amp'
    abf = abf_or_name(abf)
    axs_2_right = []
    # plot sweeps and command (single channel)


    discretized_cmap = matplotlib.cm.get_cmap('viridis', len(abf.sweepList))

    num_ch = 1;
    if len(abf.channelList)>1:
        num_ch = 2
    fig, axs = plt.subplots(num_ch, figsize = np.array(figsize)*np.array([1,num_ch]))
    if num_ch ==1:
        axs=[axs]
    theta, offset, correct_ch1 = predict_telegraph(abf)
    if num_ch==2:
        axs1_r = axs[1].twinx()
        axs[1].set_ylabel(str('Ch1 '+str(abf.sweepLabelC)))
        axs1_r.set_ylabel(str('Corrrected '+str(abf.sweepLabelC)[-4:]) )
        axs[1].set_title(str(theta) +','+ str(offset))
        axs[1].set_title('Command Correction')
    axs0_r = axs[0].twinx()
    for sweep in abf.sweepList:
        abf.setSweep(sweepNumber=sweep)
        axs[0].plot(abf.sweepX, abf.sweepY,color=discretized_cmap(sweep))        
        axs0_r.plot(abf.sweepX, abf.sweepC,color='grey' )#discretized_cmap(sweep),linestyle='dotted')
        com = abf.sweepC
        if len(abf.channelList)>1:
            abf.setSweep(sweepNumber=sweep, channel=1)
            axs[1].plot(abf.sweepX, abf.sweepY,'m')
            corrected_com = correct_ch1(theta,abf.sweepY)
            axs1_r.plot(abf.sweepX, corrected_com,'c',linestyle='dotted')
            # axs_2.set_title( str(theta)+','+ str(correct_ch1(theta,abf.sweepY[0])) )
    axs[0].set_title(abf.abfID)
    abf.setSweep(0)
    axs[0].set_ylabel(str(abf.sweepLabelY))
    axs[0].set_xlabel(str(abf.sweepLabelX))
    axs0_r.set_ylabel(str(abf.sweepLabelC))
    axs0_r.set_xlabel(str(abf.sweepLabelX))

    cmap = matplotlib.cm.get_cmap('Dark2')
    color_num=0
    for w in windows:
        color_num+=1
        rgba = cmap(color_num/len(windows))
        ylim = axs[0].get_ylim()
        sr = np.mean(np.diff( w ))
        gaps = np.where(np.diff(w)>sr*4)[0]
        sw = flatten([w[0],[ list(w[[i,i+1]]) for i in np.arange(len(w)-1) if np.diff(w)[i] > sr*3 ] , w[-1] ])
        for i in range(0, len(sw), 2):
            axs[0].axvspan(sw[i], sw[i+1], ylim[0], ylim[1], alpha=0.2,color=rgba)


    axs[0].set_zorder(1)  # default zorder is 0 for ax1 and ax2
    axs[0].patch.set_visible(False)  # prevents ax1 from hiding ax2

    fig.tight_layout()
    plt.show()
    return fig, axs , theta